In [1]:
import yfinance as yf
import pandas as pd

# Download data for PLTR
data = yf.download('PLTR', period='30d', interval='5m')

# Flatten columns if MultiIndex
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

# Convert timezone from UTC to US/Eastern (don't localize since it's already tz-aware)
data.index = data.index.tz_convert('US/Eastern')

# Remove timezone information from datetime index
data.index = data.index.tz_localize(None)

# Reset index to move datetime into a column
data.reset_index(inplace=True)

# Save to CSV
data.to_csv('PLTR_5m_30d.csv', index=False)

print("Data converted to Eastern Time, timezone removed, and saved to PLTR_5m_30d.csv")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Data converted to Eastern Time, timezone removed, and saved to PLTR_5m_30d.csv


In [3]:
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt

# 1. Clean CSV Data with Invalid Dates
def filter_invalid_data(csv_file):
    df = pd.read_csv(csv_file)
    df['Datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')
    df = df.dropna(subset=['Datetime'])
    cleaned_csv_file = 'cleaned_' + csv_file
    df.to_csv(cleaned_csv_file, index=False)
    return cleaned_csv_file

# 2. Clean and Load Data
cleaned_csv = filter_invalid_data('PLTR_5m_30d.csv')

data = bt.feeds.GenericCSVData(
    dataname=cleaned_csv,
    dtformat='%Y-%m-%d %H:%M:%S',  # Adjusted to your data's format
    datetime=0,
    close=1,
    high=2,
    low=3,
    open=4,
    volume=5,
    openinterest=-1,
    header=True,
    skiprows=1,
    timeframe=bt.TimeFrame.Minutes,
    compression=5
)

# 3. Define Strategy
class BuySellStrategy(bt.Strategy):
    params = (
        ('ema_period', 60),
        ('rsi_period', 14),
        ('atr_period', 14),
        ('rsi_threshold', 10),
        ('transaction_cost', 0.001),
    )

    def __init__(self):
        self.ema = bt.indicators.ExponentialMovingAverage(self.data.close, period=self.params.ema_period)
        self.rsi = bt.indicators.RelativeStrengthIndex(period=self.params.rsi_period)
        self.atr = bt.indicators.AverageTrueRange(self.data, period=self.params.atr_period)

        self.in_position = False
        self.last_date = None
        self.yesterday_close = None
        self.yesterday_low = None
        self.yesterday_high = None
        self.low_data = []
        self.high_data = []

        self.transaction_count = 0
        self.last_buy_price = None
        self.last_sell_price = None
        self.trade_log = []
        self.total_profit_loss = 0
        self.total_percentage_return = 0
        self.starting_cash = None

    def notify_order(self, order):
        if order.status == order.Completed:
            if order.isbuy():
                log = f"BUY EXECUTED: {order.executed.price} on {self.data.datetime.datetime(0)}"
                print(log)
                self.trade_log.append(log)
                self.last_buy_price = order.executed.price
            elif order.issell():
                log = f"SELL EXECUTED: {order.executed.price} on {self.data.datetime.datetime(0)}"
                print(log)
                self.trade_log.append(log)
                if self.last_buy_price:
                    buy_price = self.last_buy_price
                    sell_price = order.executed.price
                    transaction_cost = self.params.transaction_cost * (buy_price + sell_price)

                    capital_used = order.executed.size * buy_price
                    raw_profit = order.executed.size * (sell_price - buy_price)
                    net_profit = raw_profit - transaction_cost

                    self.total_profit_loss += net_profit
                    pct_return = (net_profit / capital_used) * 100
                    self.total_percentage_return += pct_return

                    print(f"Profit for trade: {net_profit:.2f}, Return: {pct_return:.2f}%")

                self.transaction_count += 1

    def stop(self):
        final_value = self.broker.get_value()
        pct_change = ((final_value - self.starting_cash) / self.starting_cash) * 100

        print(f"\n--- Strategy Summary ---")
        print(f"Starting Cash: {self.starting_cash}")
        print(f"Ending Cash: {self.broker.get_cash()}")
        print(f"Portfolio Value: {final_value}")
        print(f"Total Profit/Loss: {self.total_profit_loss:.2f} units")
        print(f"Total % Return from trades: {self.total_percentage_return:.2f}%")
        print(f"Overall Portfolio % Change: {pct_change:.2f}%")
        print(f"Transactions: {self.transaction_count}")
        print("Trade Log:")
        for log in self.trade_log:
            print(log)

    def next(self):
        if self.starting_cash is None:
            self.starting_cash = self.broker.get_cash()

        current_date = self.data.datetime.date(0)

        if self.last_date != current_date:
            if self.low_data and self.high_data:
                self.yesterday_close = self.data.close[-1]
                self.yesterday_low = min(self.low_data)
                self.yesterday_high = max(self.high_data)
                print(f"New day! Yesterday - Close: {self.yesterday_close}, Low: {self.yesterday_low}, High: {self.yesterday_high}")
            else:
                self.yesterday_close = self.yesterday_low = self.yesterday_high = None
                print("New day but no valid data for previous day.")

            self.low_data = []
            self.high_data = []
            self.last_date = current_date

        self.low_data.append(self.data.low[0])
        self.high_data.append(self.data.high[0])

        rsi_change = self.rsi[0] - self.rsi[-1]
        atr_value = self.atr[0]

        # ✅ **Fix: Always define midpoint**
        if self.yesterday_high is not None and self.yesterday_low is not None:
            midpoint = (self.yesterday_high + self.yesterday_low) / 2
        else:
            print(f"Skipping trade - No valid midpoint for {self.data.datetime.date(0)}")
            return  # Skip trading for this day if there's no valid midpoint

        # Buy condition
        if not self.in_position:
            if (self.data.close[0] <= self.ema[0] and
                (((self.data.close[0] <= midpoint and
                ((self.yesterday_high - self.yesterday_low) / self.yesterday_low * 100) >= 5)) or
                (self.last_sell_price is None or self.data.close[0] < self.last_sell_price)) and
                rsi_change >= self.params.rsi_threshold):

                buy_price = self.data.close[0] + (0.75 * atr_value)
                cash_available = self.broker.get_cash()
                size = cash_available / buy_price
                self.buy(size=size)
                self.in_position = True
                self.last_buy_price = buy_price  # ✅ Set last buy price for future sell logic
                print(f"Buying at {buy_price} on {self.data.datetime.datetime(0)}")

        # Sell condition
        if self.in_position and self.last_buy_price:
            stop_loss_price = self.last_buy_price * (1 - 0.07)
            take_profit_price = self.last_buy_price * (1 + 0.20)

            if self.data.close[0] <= stop_loss_price:
                size = self.position.size
                self.sell(size=size)
                self.in_position = False
                self.last_sell_price = self.data.close[0]
                self.last_buy_price = None
                print(f"Stop Loss triggered! Selling all at {self.data.close[0]} on {self.data.datetime.datetime(0)}")
                return

            if self.data.close[0] >= take_profit_price:
                size = self.position.size
                self.sell(size=size)
                self.in_position = False
                self.last_sell_price = self.data.close[0]
                self.last_buy_price = None
                print(f"Take Profit hit! Selling all at {self.data.close[0]} on {self.data.datetime.datetime(0)}")
                return

            if (self.data.close[0] >= self.ema[0] and
                (((self.data.close[0] >= midpoint and
                ((self.yesterday_high-self.yesterday_low)/self.yesterday_low*100) >= 5)) or
                self.data.close[0] > self.last_buy_price) and
                rsi_change >= self.params.rsi_threshold):

                sell_price = self.data.close[0] - (0.75 * atr_value)
                size = self.position.size
                self.sell(size=size)
                self.in_position = False
                self.last_sell_price = sell_price
                self.last_buy_price = None
                print(f"Selling at {sell_price} on {self.data.datetime.datetime(0)}")

# 4. Run Backtrader
cerebro = bt.Cerebro()
cerebro.addstrategy(BuySellStrategy)
cerebro.adddata(data)
cerebro.broker.set_cash(100000)
cerebro.broker.setcommission(commission=0.001)

print(f"Starting cash: {cerebro.broker.get_cash()}")
cerebro.run()

# 5. Plot and Save
plots = cerebro.plot(style='candlestick')
fig = plots[0][0]
fig.set_size_inches(18, 9)
fig.savefig('backtrader_plot_strategy_2_PLTR.png', bbox_inches='tight')
plt.close(fig)

# 6. Final Value
print(f"Ending cash: {cerebro.broker.get_cash()}")
print(f"Portfolio value: {cerebro.broker.get_value()}")

Starting cash: 100000
New day but no valid data for previous day.
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint for 2025-03-14
Skipping trade - No valid midpoint

<IPython.core.display.Javascript object>

Ending cash: 113759.87868218703
Portfolio value: 113759.87868218703
